# Running in the title
Exploring Spotify playlists with the word "running" in the title

In [1]:
# pip install spotify
# pip install base64
# pip install lyricsgenius

import spotify.sync as spotify
import base64
# import datetime
import requests
from urllib.parse import urlencode

In [2]:
# setting up spotify credetnials
spotify_client_id = 'f004c13efcfe46fbafedccbfbf106f67'
spotify_client_secret = 'd8de81377b164216b449bbc77ccb29b9'
spotify_client_creds = f'{spotify_client_id}:{spotify_client_secret}'
spotify_client_creds_b64 = base64.b64encode(spotify_client_creds.encode())
    
# using client ID and secret to get a token to be used for each requests
spotify_token_url = 'https://accounts.spotify.com/api/token' # https://developer.spotify.com/documentation/general/guides/authorization-guide/#client-credentials-flow
spotify_token_data = {
    'grant_type': 'client_credentials'
}

spotify_token_headers = {
    'Authorization': f"Basic {spotify_client_creds_b64.decode()}" #<base64 encoded client_id:client_secret>
}
# get the token with a POST request
r = requests.post(spotify_token_url, data=spotify_token_data, headers=spotify_token_headers)
valid_request = r.status_code in range(200, 299)
if valid_request:
#     now = datetime.datetime.now
    spotify_access_token = r.json()['access_token']
#     expires_in = r.json()['expires_in'] # in seconds
#     expires = now + datetime.timedelta(seconds=expires_in)
#     did_expire = expires < now

# API call from https://developer.spotify.com/documentation/web-api/reference/#category-search
search_item = "Running"
type_to_search = "playlist"
spotify_headers = {
    "Authorization": f"Bearer {spotify_access_token}"
}
endpoint = 'https://api.spotify.com/v1/search'
data = urlencode({"q": search_item, "type": type_to_search, "limit":1, "offset":0})

lookup_url = f"{endpoint}?{data}"

spotify_single_playlist = requests.get(lookup_url, headers=spotify_headers)

print(spotify_single_playlist.status_code)
print(spotify_single_playlist.json())

200
{'playlists': {'href': 'https://api.spotify.com/v1/search?query=Running&type=playlist&offset=0&limit=1', 'items': [{'collaborative': False, 'description': 'Run with running music hits in your workout. Want chill? See our <a href="https://open.spotify.com/playlist/4SAp2pqkCTYnu3midNetQ7?si=uyD7Y6jDSoaSFAVDP_o10Q">Chill House Hits</a> playlist!', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/0JTaSx9jkW1saMOc6t0vIk'}, 'href': 'https://api.spotify.com/v1/playlists/0JTaSx9jkW1saMOc6t0vIk', 'id': '0JTaSx9jkW1saMOc6t0vIk', 'images': [{'height': None, 'url': 'https://i.scdn.co/image/ab67706c0000bebb006f1a4f06c6c4f34a9d4608', 'width': None}], 'name': 'RUNNING Music Hits 2021 🏃\u200d♂️', 'owner': {'display_name': 'OneSevenMusic', 'external_urls': {'spotify': 'https://open.spotify.com/user/hbhz741mqsyfv0zafwby1nbq0'}, 'href': 'https://api.spotify.com/v1/users/hbhz741mqsyfv0zafwby1nbq0', 'id': 'hbhz741mqsyfv0zafwby1nbq0', 'type': 'user', 'uri': 'spotify:user:hbhz741mqsyfv0zaf

In [3]:
spotify_playlist_id = spotify_single_playlist.json()["playlists"]["items"][0]["id"]
endpoint = f'https://api.spotify.com/v1/playlists/{spotify_playlist_id}/tracks'
data = urlencode({"market": "IT", "limit":10, "offset":0, "fields": "items(track(name, album(artists)))"})
lookup_url = f"{endpoint}?{data}"

spotify_tracks_playlist = requests.get(lookup_url, headers=spotify_headers)
# print(spotify_tracks_playlist.status_code)
# print(spotify_tracks_playlist.json())

# # storing songs titles and artists within playlist
songs = [{','.join([song['track']['album']['artists'][index]['name'] for index in range(len(song['track']['album']['artists']))]): song['track']['name']} for song in spotify_tracks_playlist.json()['items']]
songs

[{'Riton,Nightcrawlers,Mufasa & Hypeman': 'Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit'},
 {'MOLOW,Rompasso': 'Bass Goes Down'},
 {'Joel Corry,RAYE,David Guetta': 'BED'},
 {'WISEKIDS,Paolo Pellegrino,Shibui': 'DESTINY (I.O.U) (feat. Shibui)'},
 {'Gabry Ponte,Lucky Luke,Kevin Palms': 'Going Down'},
 {'Majestic,Boney M.': 'Rasputin'},
 {'Tom Enzy,Sam Welch': 'Sex on Fire'},
 {'SØLO,Kush Kush,Gabry Ponte': 'All That She Wants'},
 {'Tiësto': 'The Business'},
 {'Axel Johansson,Amanda Collis': 'Always Home'}]

In [4]:
# with only one artist name when multiple
songs = [
    {song['track']['album']['artists'][0]['name']: song['track']['name']} 
    for song in spotify_tracks_playlist.json()['items']
]

In [14]:
songs

[{'Riton': 'Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit'},
 {'MOLOW': 'Bass Goes Down'},
 {'Joel Corry': 'BED'},
 {'WISEKIDS': 'DESTINY (I.O.U) (feat. Shibui)'},
 {'Gabry Ponte': 'Going Down'},
 {'Majestic': 'Rasputin'},
 {'Tom Enzy': 'Sex on Fire'},
 {'SØLO': 'All That She Wants'},
 {'Tiësto': 'The Business'},
 {'Axel Johansson': 'Always Home'}]

In [155]:
# only one
(artist_name_spotify, song_title_spotify), = songs[0].items()
print(artist_name_spotify)
print(song_title_spotify)

Riton
Friday (feat. Mufasa & Hypeman) - Dopamine Re-Edit


### check the expiration time if involving everything in a for loop (pay attention to offset param)

In [156]:
import lyricsgenius
genius_token = 'gIPW9PqAO8nW3MO8kGqJouKyx4iWRJkLcuM2LThtRtlj8z557KKj21jwIymggQdr'
genius = lyricsgenius.Genius(genius_token) # before it was api
song = genius.search_songs(song_title_spotify, per_page = 50, page=1) # possible limitations on the number of objects retrieved
artist_name_genius = [song['hits'][song_num]['result']['primary_artist']['name'] 
                      for song_num in range(len(song['hits']))]
song_title_genius = [song['hits'][song_num]['result']['title']
                      for song_num in range(len(song['hits']))]

print(f'Artists: {artist_name_genius}')
print(f'\nSongs titles: {song_title_genius}')

Artists: ['Spotify', 'Spotify', 'Spotify', 'Spotify', 'Spotify', 'Spotify', 'Riton & Nightcrawlers']

Songs titles: ['Today’s Top Hits 6/4/21 (feat. Billie Eilish)', 'Today’s Top Hits 3/5/21 (feat. Selena Gomez)', 'Today’s Top Hits 3/28/21 (feat. Cardi B)', 'Today’s Top Hits 5/14/21 (feat. The Weeknd & Ariana Grande)', 'Today’s Top Hits 5/21/21 (feat. BTS)', 'Today’s Top Hits 5/28/21 (feat. Olivia Rodrigo)', 'Friday (Dopamine Re-Edit)']


In [ ]:
# import os
# print(os.getcwd())
# artist.save_lyrics() 

In [ ]:
# import pandas as pd
# import json
# # Artist=pd.read_json(f"Lyrics_{artist_name}.json", orient='values')
# with open(f"Lyrics_{artist_name}.json") as json_data:
#     Artist_dict = json.load(json_data)
# for i in Artist_dict['songs'][0].keys():
#     print(i)

In [118]:
# #Create an empty dictionary to store your songs and related data
# artist_dict = {}
# def collectSongData(adic):
#     dps = list()
#     title = adic['title'] #song title
#     url = adic['url'] #genius url
#     artist = adic['artist'] #artist name(s)
#     song_id = adic['id'] #genius id
#     lyrics = adic['lyrics'] #song lyrics
#     year = adic['release_date'] #release date
#     upload_date = adic['description_annotation']['annotatable']['client_timestamps']['lyrics_updated_at'] #lyrics upload date
#     annotations = adic['annotation_count'] #total no. of annotations
#     descr = adic['description'] #song descriptions
    
#     dps.append((title,url,artist,song_id,lyrics,year,upload_date,annotations,descr)) #append all to one tuple list
#     artist_dict[title] = dps #assign list to song dictionary entry named after song title
    
# collectSongData(Artist_dict['songs'][0]) #check function works
# artist_dict

In [106]:
# Searching a song with combination of song_title and artist_name from spotify
# song= genius.search_song(song_title, artist_name)
# print(song.title)